In [206]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/language-identification-datasst/dataset.csv


In [207]:
df = pd.read_csv('/kaggle/input/language-identification-datasst/dataset.csv')

df.head()

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch


## understand the data and asking the big question

In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      22000 non-null  object
 1   language  22000 non-null  object
dtypes: object(2)
memory usage: 343.9+ KB


great No null values 

## basic univariate analysis

In [209]:
df['language'].value_counts()

Estonian      1000
Swedish       1000
English       1000
Russian       1000
Romanian      1000
Persian       1000
Pushto        1000
Spanish       1000
Hindi         1000
Korean        1000
Chinese       1000
French        1000
Portugese     1000
Indonesian    1000
Urdu          1000
Latin         1000
Turkish       1000
Japanese      1000
Dutch         1000
Tamil         1000
Thai          1000
Arabic        1000
Name: language, dtype: int64

In [210]:
df.duplicated().sum()

141

In [211]:
df = df.drop_duplicates()

prety good no class impalance 

here is the plan <br> we will devide the data into train, test and val set <br>
clean the data <br>
we wil start with basic machine learning models and end endup with a tranformer net from scratch to test our ability to write a full transformer from the 'Attention is all you need'. <br>
lets dive into it ... <br>

In [212]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import re
from sklearn.metrics import classification_report, accuracy_score

In [213]:
X = df['Text']
y = df['language']

In [214]:
from sklearn.base import BaseEstimator, TransformerMixin

class CleanChineseAndJapanese(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X):
        return self
    
    def transform(self, X):
        
        data = X['Text'][X['language'].isin(['Chinese', 'Japanese'])]
        data = data.apply(lambda x: self.remove_en(x))
        X['Text'][X['language'].isin(['Chinese', 'Japanese'])] = data
        
        return X
        
    def remove_en(self, x):
        pattern = r'[a-zA-Z0-9]'
        x = re.sub(pattern, '', x)
#         new_x = []
#         for w in x:
#             if w != " ":
#                 new_x.append(w)
#                 new_x.append(' ')

        return "".join(x)

In [215]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        
        return X[self.columns]

In [216]:
class TargetEncoding(BaseEstimator, TransformerMixin):
    def __init__(self, label_encoder):
        self.encoder = label_encoder
    
    def fit(self, X, y=None):
        self.encoder.fit(X)
        
        return self
    
    def transform(self, X):
        labels = np.array(X)
        unknown_mask = ~np.isin(labels, self.encoder.classes_)
        labels[unknown_mask] = 'unknown'
        return np.expand_dims(self.encoder.transform(labels), axis=1)


In [217]:
#numerical pipeline

target_pipeline = Pipeline([
    ('cols', DataFrameSelector(['language'])),
    ('lbl_encoder', TargetEncoding(LabelEncoder()))
])

target_pipeline.fit_transform(df)

array([[ 4],
       [17],
       [19],
       ...,
       [16],
       [ 1],
       [14]])

In [218]:
#categorical encoder

text_pipeline = Pipeline([
    ('clean', CleanChineseAndJapanese()),
    ('cols', DataFrameSelector(['Text'])),
])

text_pipeline.fit_transform(df).head()

,Text
0,klement gottwaldi surnukeha palsameeriti ning ...
1,sebes joseph pereira thomas på eng the jesuit...
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...
4,de spons behoort tot het geslacht haliclona en...


In [219]:
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion([
    ('text', text_pipeline),
    ('target', target_pipeline)
])

data = full_pipeline.fit_transform(df)

In [220]:
le = LabelEncoder().fit(df['language'])

In [221]:
from sklearn.model_selection import train_test_split

X = data[:, 0]
y = df['language']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.2)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=.5)

In [222]:
y_train = le.transform(y_train)
y_test = le.transform(y_test)
y_val = le.transform(y_val)

In [223]:
tfidf = TfidfVectorizer()
tfidf.fit(X)

x_train_vec = tfidf.transform(x_train)
x_test_vec = tfidf.transform(x_test)
x_val_vec = tfidf.transform(x_val)

In [224]:
svc = SVC(kernel='rbf', C=10)
svc.fit(x_train_vec, y_train)
svc_prediction = svc.predict(x_test_vec)
print(f'accuracy {accuracy_score(y_test, svc_prediction)}')

accuracy 0.9373284537968893


In [225]:
classification_report(y_test, svc_prediction)

'              precision    recall  f1-score   support\n\n           0       1.00      0.96      0.98       103\n           1       0.53      0.98      0.69       124\n           2       1.00      0.98      0.99        96\n           3       0.81      0.99      0.89        85\n           4       1.00      0.94      0.97        98\n           5       0.98      0.99      0.99       107\n           6       1.00      0.95      0.97       110\n           7       1.00      0.97      0.98        97\n           8       0.95      0.73      0.82       118\n           9       1.00      0.93      0.96        73\n          10       0.99      0.85      0.91        91\n          11       1.00      0.98      0.99        92\n          12       0.99      0.96      0.97        99\n          13       1.00      0.93      0.96       102\n          14       1.00      0.95      0.97        92\n          15       1.00      0.86      0.93        96\n          16       1.00      0.97      0.98        86\n       

In [226]:
import joblib

joblib.dump(svc, 'svc.pkl')

['svc.pkl']

In [227]:
lr = LogisticRegression(multi_class='multinomial')
lr.fit(x_train_vec, y_train)
lr_prediction = lr.predict(x_test_vec)
print(f'accuracy {accuracy_score(y_test, lr_prediction)}')

accuracy 0.9533394327538883


In [228]:
classification_report(y_test, lr_prediction)

'              precision    recall  f1-score   support\n\n           0       1.00      0.98      0.99       103\n           1       0.63      0.99      0.77       124\n           2       1.00      0.98      0.99        96\n           3       0.79      0.99      0.88        85\n           4       0.98      0.94      0.96        98\n           5       0.98      1.00      0.99       107\n           6       1.00      0.95      0.98       110\n           7       1.00      0.99      0.99        97\n           8       0.97      0.73      0.83       118\n           9       1.00      0.97      0.99        73\n          10       1.00      0.90      0.95        91\n          11       1.00      0.98      0.99        92\n          12       0.99      0.95      0.97        99\n          13       1.00      0.92      0.96       102\n          14       1.00      0.95      0.97        92\n          15       1.00      0.93      0.96        96\n          16       1.00      0.99      0.99        86\n       

In [229]:
joblib.dump(lr, 'lr.pkl')

['lr.pkl']

In [230]:
cv = CountVectorizer()
cv.fit(X)

x_train_vec = cv.transform(x_train)
x_test_vec = cv.transform(x_test)
x_val_vec = cv.transform(x_val)

In [231]:
svc = SVC(kernel='rbf', C=10)
svc.fit(x_train_vec, y_train)
svc_prediction = svc.predict(x_test_vec)
print(f'accuracy {accuracy_score(y_test, svc_prediction)}')

accuracy 0.9117108874656907


In [232]:
classification_report(y_test, svc_prediction)

'              precision    recall  f1-score   support\n\n           0       1.00      0.94      0.97       103\n           1       0.59      0.65      0.62       124\n           2       1.00      0.96      0.98        96\n           3       0.81      0.94      0.87        85\n           4       0.98      0.94      0.96        98\n           5       0.98      0.98      0.98       107\n           6       1.00      0.94      0.97       110\n           7       1.00      0.93      0.96        97\n           8       0.68      0.85      0.75       118\n           9       0.91      0.96      0.93        73\n          10       0.96      0.86      0.91        91\n          11       1.00      0.90      0.95        92\n          12       0.99      0.92      0.95        99\n          13       0.64      0.94      0.76       102\n          14       1.00      0.96      0.98        92\n          15       1.00      0.85      0.92        96\n          16       1.00      0.92      0.96        86\n       

In [233]:
joblib.dump(svc, 'svc_cnt_vct.pkl')

['svc_cnt_vct.pkl']

In [234]:
lr = LogisticRegression(multi_class='multinomial')
lr.fit(x_train_vec, y_train)
lr_prediction = lr.predict(x_test_vec)
print(f'accuracy {accuracy_score(y_test, lr_prediction)}')

accuracy 0.9391582799634035


In [235]:
classification_report(y_test, lr_prediction)

'              precision    recall  f1-score   support\n\n           0       1.00      0.97      0.99       103\n           1       0.95      0.48      0.64       124\n           2       1.00      0.99      0.99        96\n           3       0.85      1.00      0.92        85\n           4       0.98      0.96      0.97        98\n           5       0.98      1.00      0.99       107\n           6       1.00      0.95      0.98       110\n           7       1.00      0.99      0.99        97\n           8       0.52      0.97      0.68       118\n           9       1.00      0.93      0.96        73\n          10       0.98      0.88      0.92        91\n          11       1.00      0.99      0.99        92\n          12       0.99      0.96      0.97        99\n          13       1.00      0.95      0.97       102\n          14       1.00      0.99      0.99        92\n          15       0.99      0.89      0.93        96\n          16       1.00      0.99      0.99        86\n       

In [236]:
joblib.dump(lr, 'lr_cnt_vct.pkl')

['lr_cnt_vct.pkl']

logistic regression is performing better than support vector machines  
and tfidf vectoriver is performing better than count vectorizer  
there is some confusion between chinese and japanese  
lets solve this problem using nueral network

# Using RNN

In [49]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [37]:
import string

class RemovePunctiuation(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['Text'] = X['Text'].apply(lambda row: self.remove_punc(row))
        
        return X
    
    def remove_punc(self, row):
        translator = str.maketrans('', '', string.punctuation)
        
        return row.translate(translator)
        
df = RemovePunctiuation().fit_transform(df) 

In [190]:
df = CleanChineseAndJapanese().fit_transform(df)
X = df['Text']
y = df['language']

In [191]:
X

0        klement gottwaldi surnukeha palsameeriti ning ...
1        sebes joseph pereira thomas  på eng the jesuit...
2        ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...
3        விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...
4        de spons behoort tot het geslacht haliclona en...
                               ...                        
21995    hors du terrain les années  et  sont des année...
21996    ใน พศ  หลักจากที่เสด็จประพาสแหลมมลายู ชวา อินเ...
21997    con motivo de la celebración del septuagésimoq...
21998    年月，當時還只有歲的她在美國出道，以-名義推出首張英文《  》，由美國的獨立廠牌·發行，以外...
21999     aprilie sonda spațială messenger a nasa și-a ...
Name: Text, Length: 21859, dtype: object

In [192]:
tok = Tokenizer(oov_token='[UNK]')
tok.fit_on_texts(X)

In [195]:
max_length = max(len(s.split()) for s in X)
vocab_size = len(tok.word_index) + 1
num_classes = len(y.unique())

print(f'length of vocab : {vocab_size}')
print(f'max length : {max_length}')  
print(f'num_classes : {num_classes}')

length of vocab : 273550
max length : 331
num_classes : 22


In [196]:
x_train_padded = pad_sequences(tok.texts_to_sequences(x_train), maxlen=max_length)
x_test_padded = pad_sequences(tok.texts_to_sequences(x_test), maxlen=max_length)
x_val_padded = pad_sequences(tok.texts_to_sequences(x_val), maxlen=max_length)

In [197]:
config = {
    'embd_size': 100,
    'batch_size': 128,
    'epochs': 10
}

In [198]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding, Input, Flatten, Bidirectional
from tensorflow.keras.regularizers import L2

model = Sequential()
model.add(Embedding(input_dim=vocab_size+1,
                    output_dim=config['embd_size'],
                    input_length=max_length,
                    embeddings_regularizer=L2(.01)))
model.add(Bidirectional(GRU(100, return_sequences=True)))
model.add(GRU(100))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(22, activation='softmax'))

model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 331, 100)          27355100  
                                                                 
 bidirectional_5 (Bidirectio  (None, 331, 200)         121200    
 nal)                                                            
                                                                 
 gru_17 (GRU)                (None, 100)               90600     
                                                                 
 dense_61 (Dense)            (None, 256)               25856     
                                                                 
 dense_62 (Dense)            (None, 128)               32896     
                                                                 
 dense_63 (Dense)            (None, 64)                8256      
                                                     

In [199]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train_padded, y_train))\
                          .batch(config['batch_size'])\
                          .cache()\
                          .prefetch(config['batch_size'])

test_ds = tf.data.Dataset.from_tensor_slices((x_test_padded, y_test))\
                          .batch(config['batch_size'])\
                          .cache()\
                          .prefetch(config['batch_size'])

val_ds = tf.data.Dataset.from_tensor_slices((x_val_padded, y_val))\
                          .batch(config['batch_size'])\
                          .cache()\
                          .prefetch(config['batch_size'])

In [200]:
for i in train_ds.take(1).as_numpy_iterator():
    print(i)

(array([[     0,      0,      0, ..., 231232, 231233, 231234],
       [     0,      0,      0, ...,  34527,   9408,    341],
       [     0,      0,      0, ...,      3,   6189,   3995],
       ...,
       [     0,      0,      0, ...,  25801,  82898, 175870],
       [     0,      0,      0, ...,    645,  13178, 185580],
       [     0,      0,      0, ..., 123438,    533,  24471]], dtype=int32), array([19, 21,  3, 13, 13, 17, 11,  6, 21,  7,  1, 21, 18, 20, 19,  1, 14,
       18, 16,  3, 20, 14, 18,  4, 18, 21, 16,  0, 10, 21, 21,  5, 11,  5,
       10, 12,  2, 15, 10, 15, 15, 21, 20,  6,  4, 17, 15, 10,  0, 10, 17,
       14, 19, 12, 21,  3, 19, 10, 12,  7,  0, 20, 19, 17,  5, 10, 18, 14,
       19,  4,  2, 20,  6,  7,  6,  2,  5,  9, 10, 19, 12,  9, 11, 18,  6,
        8,  7,  9, 15, 18, 20, 21, 14,  6, 16,  2, 21,  7, 17, 14, 11,  7,
        6, 10,  3, 16,  7, 10, 19, 15,  3,  4,  4, 17, 21, 13,  1, 15, 11,
       14,  1, 15,  5, 15, 13, 15, 15, 16]))


In [201]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [202]:
model.fit(train_ds,
         validation_data=(val_ds),
         epochs=config['epochs'])

Epoch 1/10
137/137 [==============================] - 17s 81ms/step - loss: 25.7175 - acc: 0.2532 - val_loss: 2.2421 - val_acc: 0.4071
Epoch 2/10
137/137 [==============================] - 10s 74ms/step - loss: 2.0662 - acc: 0.5377 - val_loss: 1.8792 - val_acc: 0.6958
Epoch 3/10
137/137 [==============================] - 10s 73ms/step - loss: 1.5950 - acc: 0.7908 - val_loss: 1.5267 - val_acc: 0.8134
Epoch 4/10
137/137 [==============================] - 10s 73ms/step - loss: 1.4304 - acc: 0.8486 - val_loss: 1.3135 - val_acc: 0.8532
Epoch 5/10
137/137 [==============================] - 10s 74ms/step - loss: 1.2313 - acc: 0.8815 - val_loss: 1.2473 - val_acc: 0.8861
Epoch 6/10
137/137 [==============================] - 10s 74ms/step - loss: 1.0456 - acc: 0.9052 - val_loss: 1.0259 - val_acc: 0.8911
Epoch 7/10
137/137 [==============================] - 10s 73ms/step - loss: 1.0663 - acc: 0.9020 - val_loss: 0.9804 - val_acc: 0.9012
Epoch 8/10
137/137 [==============================] - 10s 74m

In [203]:
tst_prds = model.predict(test_ds)
predicted_labels = le.inverse_transform(np.argmax(tst_prds, axis=1))
y_predicted = le.transform(predicted_labels)
classification_report(y_test, y_predicted)

18/18 [==============================] - 1s 26ms/step


'              precision    recall  f1-score   support\n\n           0       1.00      0.98      0.99       104\n           1       0.48      0.75      0.59        93\n           2       0.99      0.94      0.97       108\n           3       0.85      0.94      0.90       100\n           4       0.93      0.93      0.93       106\n           5       0.93      0.98      0.96       100\n           6       0.99      0.98      0.98        81\n           7       0.99      0.97      0.98        88\n           8       0.44      0.16      0.24        86\n           9       0.96      0.98      0.97        93\n          10       0.94      0.92      0.93        97\n          11       1.00      0.98      0.99        90\n          12       0.98      0.98      0.98        96\n          13       0.99      0.94      0.96       102\n          14       0.98      0.95      0.96       115\n          15       0.96      0.99      0.98       102\n          16       0.98      0.98      0.98       101\n       

In [205]:
model.save('rnn.h5')

array(['Chinese', 'Chinese', 'Swedish', ..., 'Dutch', 'Tamil', 'Russian'],
      dtype=object)